In [3]:
!pip install tsfresh

In [6]:
import pandas as pd
import numpy as np

from collect_bike_data import collect_bike_data
from aggregate_time_series import aggregate_time_series
# from tsfresh.utilities.dataframe_functions import roll_time_series, make_forecasting_frame
from tsfresh import extract_features, extract_relevant_features

# from tsfresh.feature_extraction.settings import EfficientFCParameters
# from tsfresh.utilities.dataframe_functions import impute
import xgboost as xgb


from tqdm import tqdm
from sklearn.ensemble import AdaBoostRegressor

ModuleNotFoundError: No module named 'tsfresh'

In [2]:
df = collect_bike_data('assets/bikes_database.sqlite')
df_agg = aggregate_time_series(df,'Week')
df_agg['id'] = 1

In [3]:
fresh_df = pd.concat([pd.Series(df_agg.index.values), pd.Series(df_agg['price_ext'].values), pd.Series(df_agg['id'].values)],axis=1)
fresh_df.columns = ['Time', 'value', 'id']
fresh_df['kind'] = 'bikes'
fresh_df.head()

,Time,value,id,kind
0,2011-01-09,12040.0,1,bikes
1,2011-01-16,151460.0,1,bikes
2,2011-01-23,143850.0,1,bikes
3,2011-01-30,175665.0,1,bikes
4,2011-02-06,105210.0,1,bikes


In [4]:
use_previous = 12

In [5]:
rolled_df = roll_time_series(fresh_df.copy(), column_id='id',
                             column_sort='Time',
                             column_kind='kind',
                             rolling_direction=1,
                            max_timeshift=use_previous-1)
rolled_df.head()

,Time,value,id,kind
2794,2011-01-09,12040.0,2011-01-09,bikes
2535,2011-01-09,12040.0,2011-01-16,bikes
2795,2011-01-16,151460.0,2011-01-16,bikes
2277,2011-01-09,12040.0,2011-01-23,bikes
2536,2011-01-16,151460.0,2011-01-23,bikes


In [6]:
rolled = rolled_df.groupby('id').filter(lambda x : len(set(x['Time'])) == use_previous).reset_index(drop=True)

In [7]:
# extract_features(rolled,column_id='id',
#                  column_sort='Time',
#                  column_kind='kind',
#                  column_value='value',
#                 default_fc_parameters=EfficientFCParameters(), n_jobs=6)

In [8]:
df_shift, y = make_forecasting_frame(fresh_df["value"], kind="bike", max_timeshift=10, rolling_direction=1)

In [9]:
y.head()

1    151460.0
2    143850.0
3    175665.0
4    105210.0
5    250390.0
Name: value, dtype: float64

In [37]:
df_shift.head()

,time,value,id,kind
2286,0.0,12040.0,1,bike
2028,0.0,12040.0,2,bike
2287,1.0,151460.0,2,bike
1771,0.0,12040.0,3,bike
2029,1.0,151460.0,3,bike


In [11]:
# X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,
#                      show_warnings=False)

X = extract_relevant_features(df_shift, y=y, column_id="id", column_sort="time", column_value="value")

Feature Extraction: 100%|██████████| 20/20 [00:06<00:00,  3.10it/s]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"slope"'
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"rvalue"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"slope"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_10__attr_"stderr"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"intercept"'
 'value__agg_linear_trend__f_agg_"mean"__chunk_len_50__attr_"rvalue"'
 'value__agg_linear_tre

In [12]:
# drop constant features
print(X.shape)
X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
print(X.shape)

(259, 69)
(259, 69)


In [13]:
# Add last value as feature
X["feature_last_value"] = y.shift(1)

In [14]:
# Drop first line
X = X.iloc[1:, ]
y = y.iloc[1: ]

In [15]:
X.head()

variable,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_10","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_5",...,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_0__w_10","value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_2","value__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""rvalue""",value__autocorrelation__lag_2,"value__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""intercept""","value__change_quantiles__f_agg_""var""__isabs_True__qh_0.6__ql_0.0",value__autocorrelation__lag_1,value__partial_autocorrelation__lag_1,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_1__w_5",feature_last_value
id,,,,,,,,,,,,,,,,,,,,,
2,476034.59049,286535.023123,373779.610288,438855.985666,337849.275434,390914.25783,510221.443225,547359.922266,389521.482144,583509.476594,...,3289.867623,181299.348311,-1.0,-0.171867,255139.5,0.000000e+00,-1.000000,-1.000000,62470.988378,151460.0
3,476034.59049,286535.023123,373779.610288,438855.985666,337849.275434,390914.25783,510221.443225,547359.922266,389521.482144,583509.476594,...,44222.987173,181299.348311,-1.0,-0.913670,255139.5,0.000000e+00,-0.293165,-0.293165,115646.696869,143850.0
4,476034.59049,286535.023123,373779.610288,438855.985666,337849.275434,390914.25783,510221.443225,547359.922266,389521.482144,583509.476594,...,43442.202592,181299.348311,-1.0,-0.101134,255139.5,0.000000e+00,-0.111239,-0.111239,111011.562065,175665.0
5,476034.59049,286535.023123,373779.610288,438855.985666,337849.275434,390914.25783,510221.443225,547359.922266,389521.482144,583509.476594,...,81348.897363,181299.348311,-1.0,-0.114248,255139.5,0.000000e+00,-0.142846,-0.142846,168481.228947,105210.0
6,476034.59049,286535.023123,373779.610288,438855.985666,337849.275434,390914.25783,510221.443225,547359.922266,389521.482144,583509.476594,...,79292.382562,181299.348311,1.0,0.179310,117645.0,4.343469e+09,-0.244809,-0.244809,156109.490298,250390.0


# Fit XGBoost Model 

In [32]:
regressor = xgb.XGBRegressor(max_depth=6,
                                 learning_rate=0.6,
                                 n_estimators=10,
                                 gamma=0.01)

In [33]:
y_pred = [np.NaN] * len(y)

isp = 100   # index of where to start the predictions
assert isp > 0
for i in tqdm(range(isp, len(y))):
    
    regressor.fit(X.iloc[:i], y[:i])
    y_pred[i] = regressor.predict(X.iloc[i, :].values.reshape((1, -1)))[0]
    
y_pred = pd.Series(data=y_pred, index=y.index)

  0%|          | 0/158 [00:00<?, ?it/s]


ValueError: feature_names mismatch: ['value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_20', 'value__c3__lag_1', 'value__quantile__q_0.9', 'value__mean', 'value__abs_energy', 'value__sum_values', 'value__fft_coefficient__coeff_0__attr_"abs"', 'value__fft_coefficient__coeff_0__att

In [40]:
regressor.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=1, gamma=0.01, learning_rate=0.6,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=None,
             n_estimators=10, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=True, subsample=1)

In [59]:
X.iloc[10,:].values.reshape((1, -1))

array([[ 3.73385877e+05,  2.50137282e+05,  3.15344131e+05,
         3.85531907e+05,  2.99053266e+05,  2.76749801e+05,
         4.24769115e+05,  4.57201135e+05,  3.21439262e+05,
         5.13365933e+05,  4.69500665e+05,  3.54827062e+05,
         1.01745397e+16,  3.18637500e+05,  2.01480000e+05,
         5.29382843e+11,  2.01480000e+06,  2.01480000e+06,
         2.01480000e+06,  9.09946920e+15,  2.74392000e+05,
         5.37091796e+05,  5.00573064e+05,  7.27906033e+15,
         3.81096374e+05,  2.51852000e+05,  2.57597000e+05,
         2.13027500e+05,  1.32258000e+05,  4.10595000e+05,
         1.62939000e+05,  9.52260000e+04,  4.50275135e+05,
         5.30084758e+05,  3.33829006e+05,  3.22274312e+05,
         4.30550243e+05,  5.43060000e+04,  4.54500000e+04,
        -2.96219753e+04,  1.23440939e+10,  1.11103978e+05,
         1.03302334e+00,  1.14643618e+00,  3.10415651e+05,
         4.85834657e+05,  4.02064320e+05,  1.36758445e+04,
         2.05256087e+00,  2.51196107e+05,  3.20764650e+0

In [61]:
regressor.predict(X.iloc[10,:].values.reshape((1, -1)))

ValueError: feature_names mismatch: ['value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_8__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_7__w_20', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_5', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_10', 'value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_6__w_20', 'value__c3__lag_1', 'value__quantile__q_0.9', 'value__mean', 'value__abs_energy', 'value__sum_values', 'value__fft_coefficient__coeff_0__attr_"abs"', 'value__fft_coefficient__coeff_0__att

In [63]:
X['f39']

KeyError: 'f39'

In [35]:
ada = AdaBoostRegressor(n_estimators=10, learning_rate=0.6)
y_pred = [np.NaN] * len(y)

isp = 100   # index of where to start the predictions
assert isp > 0

for i in tqdm(range(isp, len(y))):
    
    ada.fit(X.iloc[:i], y[:i])
    y_pred[i] = ada.predict(X.iloc[i, :].values.reshape((1, -1)))[0]
    
y_pred = pd.Series(data=y_pred, index=y.index)

100%|██████████| 158/158 [00:05<00:00, 28.77it/s]


In [36]:
print(pd.concat([y_pred,y], axis=1))

                 0     value
2              NaN  143850.0
3              NaN  175665.0
4              NaN  105210.0
5              NaN  250390.0
6              NaN  410595.0
7              NaN  254045.0
8              NaN  308420.0
9              NaN   45450.0
10             NaN   55290.0
11             NaN  265885.0
12             NaN  136595.0
13             NaN  476175.0
14             NaN  542175.0
15             NaN  306210.0
16             NaN  492750.0
17             NaN  125410.0
18             NaN  466260.0
19             NaN   63325.0
20             NaN  138025.0
21             NaN  211320.0
22             NaN  309455.0
23             NaN  199170.0
24             NaN  383750.0
25             NaN  367300.0
26             NaN  302395.0
27             NaN  480860.0
28             NaN  312855.0
29             NaN   91920.0
30             NaN  180140.0
31             NaN  174710.0
..             ...       ...
230  466136.600000  445400.0
231  507655.555556  405045.0
232  476528.33